# Following this blog post series:


In [1]:
# bokeh basics
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

# Create a blank figure with labels
p = figure(plot_width = 600, plot_height = 600, 
           title = 'Example Glyphs',
           x_axis_label = 'X', y_axis_label = 'Y')

# Example data
squares_x = [1, 3, 4, 5, 8]
squares_y = [8, 7, 3, 1, 10]
circles_x = [9, 12, 4, 3, 15]
circles_y = [8, 4, 11, 6, 10]

# Add squares glyph
p.square(squares_x, squares_y, size = 12, color = 'navy', alpha = 0.6)
# Add circle glyph
p.circle(circles_x, circles_y, size = 12, color = 'red')

# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

Loading BokehJS ...

In [35]:
import pandas as pd
from pathlib import Path
from bokeh.models import ColumnDataSource
from bokeh.palettes import Dark2_5 as palette
import itertools

In [36]:
# Read the data from a csv into a dataframe
flights = pd.read_csv('data/flights.csv', index_col=0)

In [65]:
p = figure(plot_width=500, plot_height=400)
colors = itertools.cycle(palette) 

temperature_arrays = []
seebeck_arrays = []
color_array = []
label_array = []

for f in Path('data/').glob('*.xlsx'):
    print(f)
    df = pd.read_excel(f)
    temperature_arrays.append(df['Temperature (K)'])
    seebeck_arrays.append(df['Seebeck (V/K)'])
    color_array.append(next(colors))
    label_array.append(f"{f}")

source = ColumnDataSource(dict(temperature=temperature_arrays, seebeck=seebeck_arrays, color=color_array,
                              label=label_array))
# p.line('Temperature (K)', 'Seebeck (V/K)', line_width=2, source=source, color=next(colors),
#       legend=f"{f}")

p.multi_line('temperature', 'seebeck', line_width=2, source=source, color='color', legend='label')

p.xaxis.axis_label = 'Temperature (K)'
p.yaxis.axis_label = 'Seebeck coefficient (V/K)'
p.legend.location = 'bottom_right'

show(p)

data/fake_netzsch_data2.xlsx
data/fake_netzsch_data1.xlsx


In [16]:
df.columns

Index(['Temperature (K)', 'Seebeck (V/K)', 'Electrical Conductivity (S)'], dtype='object')

In [38]:
for i, color in zip(range(3), colors):
    print(color)

#1b9e77
#d95f02
#7570b3


In [39]:
palette[3]

'#e7298a'

In [63]:
from collections import defaultdict

import numpy as np
from scipy.stats import norm

from bokeh.layouts import gridplot
from bokeh.models import HoverTool, TapTool
from bokeh.palettes import Viridis6
from bokeh.plotting import figure, show

mass_spec = defaultdict(list)

RT_x = np.linspace(118, 123, num=50)
norm_dist = norm(loc=120.4).pdf(RT_x)

# Generate several gaussian distributions and spectral lines
for scale, mz in [(1.0, 83), (0.9, 55), (0.6, 98), (0.4, 43), (0.2, 39), (0.12, 29)]:
    mass_spec["RT"].append(RT_x)
    mass_spec["RT_intensity"].append(norm_dist * scale)
    mass_spec["MZ"].append([mz, mz])
    mass_spec["MZ_intensity"].append([0, scale])
    mass_spec['MZ_tip'].append(mz)
    mass_spec['Intensity_tip'].append(scale)

mass_spec['color'] = Viridis6

mass_spec = ColumnDataSource(mass_spec)

figure_opts = dict(plot_width=450, plot_height=300)
hover_opts = dict(
    tooltips=[('MZ', '@MZ_tip'), ('Rel Intensity', '@Intensity_tip')],
    show_arrow=False,
    line_policy='next'
)
line_opts = dict(
    line_width=5, line_color='color', line_alpha=0.6,
    hover_line_color='color', hover_line_alpha=1.0,
    source=mass_spec
)

rt_plot = figure(tools=[HoverTool(**hover_opts), TapTool()], **figure_opts)
rt_plot.multi_line(xs='RT', ys='RT_intensity', legend="Intensity_tip", **line_opts)
rt_plot.xaxis.axis_label = "Retention Time (sec)"
rt_plot.yaxis.axis_label = "Intensity"

mz_plot = figure(tools=[HoverTool(**hover_opts), TapTool()], **figure_opts)
mz_plot.multi_line(xs='MZ', ys='MZ_intensity', legend="Intensity_tip", **line_opts)
mz_plot.legend.location = "top_center"
mz_plot.xaxis.axis_label = "MZ"
mz_plot.yaxis.axis_label = "Intensity"

show(gridplot([[rt_plot, mz_plot]]))